# **PROHACK2020 , Ouss_M team. **

---




![](https://raw.githubusercontent.com/ousama-88/prohack_dataset_fIZoqt7/master/0%20(1).jpg)

---



**Our solution to the data science hackathon by McKinsey 'Prohack 2020' , team : Ouss_M , which was ranked 2nd in final leaderboard. There are 3 notebooks in this repo :**

---




1.   notebook1 : for data :EDA , the final df is the one we worked with in the second notebook
2.   notebook2 : more feature engineering + modeling " TASK 1 SOLUTION "
3.   notebook3 : " TASK 2 SOLUTION " based on the y predicted from task 1 'notebook2' 


---





 **Probelm Description**
: “Beeep…Beeeep….Beeeep… Hooomans*, are you there?...” This very strange transmission is coming from your narrowband radio signal receiver, pointed towards one of the farthest away galaxies. It’s early morning, you are sitting in your radio observatory high in the mountains. For the last 10 years you’ve been a Chief Data Scientist in one of the best astrophysics research teams in the world. You are enjoying a quiet time with a cup of coffee and reviewing the data reports from last night, when this strange sound arrived. You almost spill your coffee in surprise. “Am I dreaming?” is your first thought as you move closer towards the speaker and listen… “Beep…Beeeep….Beeeep… To all Hooomans who can hear us – we need your help” You lean closer and grab a notebook and a pencil – you don’t really trust computers when it comes to such important tasks as taking notes from a radio transmission. You start recording everything that the strange voice from light years away is saying. “… We need serious Data Science help and we know you Hooomans are the best at it…. We are an intergalactic species which have almost achieved singularity and the highest possible levels of development. We travel fast through space and explore other galaxies” “The only essence that we consume is energy, measured in DSML units…Our populace is widespread and we live across many different star clusters and galaxies. What we need now is to optimize our well-being across all those galaxies… We have a lot of data but our сomputers and methods are too weak – we urgently need your data science knowledge to help us” “Only two steps prevent us from achieving singularity · To understand what makes us better off. Our elders used the composite index to measure our well-being performance, but this knowledge has disappeared in the sands of time. Use our data and train your model to predict this index with the highest possible level of certainty. · To achieve the highest possible level of well-being through optimized allocation of additional energy We have discovered the star of an unusually high energy of 50000 zillion DSML. We have agreed between ourselves that · no one galaxy will consume more than 100 zillion DSML and at least 10% of the total energy will be consumed by galaxies in need with existence expectancy index below 0,7. Think of our galaxies as your “countries” (or how you call them??) and our population as citizens. We have similar healthcare and wellbeing characteristic as you, Hooomans” “We are sending all the data to you right now. Let the data be with you, Hoomans… … …” Transmission suddenly ends. You put your notebook and pencil away and start thinking. You really want to help this species optimize their well-being. You open up Python and upload the dataset from the narrowband radio signal receiver. It will be another great day at the observatory today. ———— * probably intergalactic species meant to say “humans” here but we will never know for sure


---



**Description Data Recieved :**
The solutions are evaluated on two criteria: predicted future Index values and allocated energy from a newly discovered star

1) Index predictions are evaluated using RMSE metric

2) Energy allocation is also evaluated using RMSE metric and has a set of known factors that need to be taken into account.

Every galaxy has a certain limited potential for improvement in the index described by the following function:

Potential for increase in the Index = -np.log(Index+0.01)+3

Likely index increase dependent on potential for improvement and on extra energy availability is described by the following function:

Likely increase in the Index = extra energy * Potential for increase in the Index **2 / 1000

There are also several constraints:

in total there are 50000 zillion DSML available for allocation and no galaxy at a point in time should be allocated more than 100 zillion DSML or less than 0 zillion DSML. Galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available in the foreseeable future

3) Leaderboard is based on a combined scaled metric:

80% prediction task RMSE + 20% optimization task RMSE * lambda where lambda is a normalizing factor

4) Leaderboard is 80% public and 20% private


---



# **this is the first notebook , how do we get our preprocessed data **


---



---



in our approache we aim to get clean scalable data as the real data corresponding to **Human Development Data (1990-2018)** , we have figured out that the data used for the problem mentioned in this hackathon is quite similar to the data collected from the real word . 

![](https://raw.githubusercontent.com/ousama-88/prohack_dataset_fIZoqt7/master/011.PNG)

more details : http://hdr.undp.org/en/data !

In [1]:
!pip install pygam catboost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import decomposition
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from scipy.stats import ttest_ind
from itertools import combinations
from sklearn.ensemble import IsolationForest
from catboost import CatBoostRegressor, Pool
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from scipy.interpolate import interp1d, splrep, splev 
from scipy.optimize import curve_fit
import time
import numba
from sklearn.svm import SVR
from tqdm import tqdm
from sklearn.base import clone

     |████████████████████████████████| 1.8MB 2.7MB/s 
     |████████████████████████████████| 64.8MB 61kB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


functions used for data processing

In [2]:
def percentil5 (x):
    x = x.dropna()
    if x.shape[0] > 0:
        return np.round(np.percentile(x, q=5), 3)
    else: return 0
def percentil95 (x): 
    x = x.dropna()
    if x.shape[0] > 0:
        return np.round(np.percentile(x, q=95), 3)
    else: return 0

def group_stat(df, group, for_stat):
    gr = df.groupby(group).agg(
        Par_min = (for_stat, 'min'),
        Par_quantil1 = (for_stat, percentil5),
        Par_median = (for_stat, 'median'),
        Par_mean = (for_stat, 'mean'),
        Par_quantil3 = (for_stat, percentil95),
        Par_max = (for_stat, 'max'),
        Par_sum = (for_stat, 'sum'),
        Par_count = (for_stat, 'count')).reset_index()
    return gr

def catBoost_rec(df, params, rec):
    model = CatBoostRegressor(iterations = 10000, learning_rate = 0.1, depth = 4,
                              custom_metric = 'R2', eval_metric = 'R2', verbose = 200)
    dfm = df[params+[rec]]
    dfm = dfm.dropna()
    X = dfm.drop([rec], axis = 1)
    y = dfm[rec]    
    X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.15)
    eval_pool = Pool(X_eval, y_eval, cat_features = [1])
    model.fit(X_train, y_train, cat_features = [1], eval_set=eval_pool, early_stopping_rounds=200)
    y_pred = model.predict(df[params])
    return y_pred

def param_int(df, col):
    dfn = df[['galactic year', col]]
    dfn = dfn.dropna()
    f = interp1d(dfn['galactic year'], dfn[col], kind='linear', 
                 bounds_error = False, assume_sorted = True, fill_value = (dfn.iloc[0,1], dfn.iloc[-1,1]))
    y2 = f(df['galactic year'])
    return y2

def func(x, a, b):     
    return a * x + b

def func_pow(x, a, b):     
    return 1 / (a + np.exp(-b * x))

def exp2(x, a, b, c):     
    return a + b * x**c

def korrect_y(df, par):
    gr = group_stat(df, ['galactic year'], par)
    y = gr['Par_mean'].median()
    for i in range(gr.shape[0]):
        if gr.loc[i, 'Par_mean'] == 0:
            koef = 1
        else:
            koef = y / gr.loc[i, 'Par_mean']
        df.loc[df['galactic year'] == gr.iloc[i, 0], par] =\
        df.loc[df['galactic year'] == gr.iloc[i, 0], par] * koef
    return df

def compute_meta_feature(model, X_train, X_test, y_train, cv, log = 0):
    X_meta_train = np.zeros((len(y_train), 1), dtype=np.float32)
    splits = cv.split(X_train)
    for train_fold_index, predict_fold_index in splits:
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(model)
        folded_clf.fit(X_fold_train, y_fold_train, cat_features = [1])
        
        X_meta_train[predict_fold_index] = folded_clf.predict(X_fold_predict).reshape(-1, 1)
    
    meta_clf = clone(model)
    meta_clf.fit(X_train, y_train, cat_features = [1])
    
    X_meta_test = meta_clf.predict(X_test).reshape(-1, 1)
    if log == 1:
        X_meta_train = np.exp(X_meta_train)
        X_meta_test = np.exp(X_meta_test)
    return X_meta_train, X_meta_test

def generate_meta_features(models, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(model, X_train, X_test, y_train, cv)
        for model in tqdm(models)
    ]
    stacked_features_train = np.hstack([
        features_train for features_train, features_test in features
    ])

    stacked_features_test = np.hstack([
        features_test for features_train, features_test in features
    ])
    
    return stacked_features_train, stacked_features_test

def compute_metric(clf, X_train, y_train, X_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def filter_iqr_dist(df, par):
    df_log = df[par].dropna()
    p125, p25, p50, p75, p875 = df_log.quantile([0.125, 0.25, 0.5, 0.75, 0.875])
    iqr = p75 - p25

    QS = ((p75 - p50) - (p50 - p25)) / (p75 - p25)
    OS = ((p875 - p50) - (p50 - p125)) / (p875 - p125)
    mc = (3 * QS + 2 * OS) / 5
    if mc >= 0:
        l = p25 - 2 * np.exp(-3.5 * mc) * iqr
        u = p75 + 2 * np.exp(4 * mc) * iqr
    else:
        l = p25 - 2 * np.exp(-4 * mc) * iqr
        u = p75 + 2 * np.exp(3.5 * mc) * iqr
    isn1 = df[df[par] <= l].index
    df.loc[isn1, par] = np.nan
    isn2 = df[df[par] >= u].index
    df.loc[isn2, par] = np.nan
    if len(list(isn1)) > 0 or len(list(isn2)) > 0:
        filter_iqr(df, par)
    return df

def filter_iqr(df, par):
    p25 = df[par].quantile(0.25)
    p75 = df[par].quantile(0.75)
    iqr = p75 - p25
    df.loc[(df[par] > p75 + 1.5 * iqr) | (df[par] < p25 - 1.5 * iqr), par] = np.nan
    return df

def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

def valid_ind(df):
    df_train = df[~df['galactic year'].isin([1007012,1008016,1009020,1010025,1011030,1012036,1013042,1014049,1015056])]
    df_test = df[df['galactic year'].isin([1007012,1008016,1009020,1010025,1011030,1012036,1013042,1014049,1015056])]
    
    df_test, df_dop = train_test_split(df_test, test_size=0.3, random_state = 0)
    
    df_train = df_train.append(df_dop[df_dop['galactic year'] < 1015056])
    df_test = df_test.append(df_dop[df_dop['galactic year'] == 1015056])
    return (df_train.index), (df_test.index)

def fill_by_RMSE(dft, df_stats_gal, par = 'y_cor', R2 = 0.97, p_val = 0.5, RMSE = 0.01, n = 3):
    ##Select the most similar and create a dictionary galaxy: similar galaxies
    df_stats_gal_f = df_stats_gal[(df_stats_gal['R2'] > R2) & (df_stats_gal['p_value'] > p_val) & (df_stats_gal['RMSE'] < RMSE)]
    galaxy_comb = dict()
    for i in range(df_stats_gal_f.shape[0]):
        if df_stats_gal_f.iloc[i, 0] in galaxy_comb:
            galaxy_comb[df_stats_gal_f.iloc[i, 0]].append([df_stats_gal_f.iloc[i, 1], df_stats_gal_f.iloc[i, 2]])
        else:
            galaxy_comb[df_stats_gal_f.iloc[i, 0]] = [[df_stats_gal_f.iloc[i, 1], df_stats_gal_f.iloc[i, 2]]]
    
    for i in range(df_stats_gal_f.shape[0]):
        if df_stats_gal_f.iloc[i, 1] in galaxy_comb:
            galaxy_comb[df_stats_gal_f.iloc[i, 1]].append([df_stats_gal_f.iloc[i, 0], df_stats_gal_f.iloc[i, 2]])
        else:
            galaxy_comb[df_stats_gal_f.iloc[i, 1]] = [[df_stats_gal_f.iloc[i, 0], df_stats_gal_f.iloc[i, 2]]]    
    
    ##We select only those where more than 3 similar
    key_to_del = []
    for key in galaxy_comb:
        if len(galaxy_comb[key]) < n:
            key_to_del.append(key)
    for key in key_to_del:
        galaxy_comb.pop(key, None)
    
    ##We fill in RMSE-weighted parameters based on the dictionary on the fact variable
    print(dft[par].isnull().sum())
    for galaxy in galaxy_comb:
        data = dft.loc[dft['galaxy'] == galaxy, ['galactic year', par]]
        ind = data.index
        sum_error = []
        i2 = 0
        for galaxy2 in galaxy_comb[galaxy]:
            data = pd.merge(data, dft.loc[dft['galaxy'] == galaxy2[0], ['galactic year', par]], 
                            on=['galactic year'], how='left', suffixes = (str(i2 * 2), str(i2 * 2 + 1)))
            data.iloc[:, -1] = data.iloc[:, -1] / galaxy2[1]
            sum_error.append(1 / galaxy2[1])
            i2 += 1
        data['mean'] = data.apply(lambda x: x[2:].sum() / np.sum(x[2:]/x[2:]*sum_error) , axis = 1)
        data.index = ind
        isn = data.iloc[:, 1]
        isn = isn[(isn.isnull()) & (data.iloc[:, 0] >=1011030)].index
        dft.loc[isn, par] = data.loc[isn, 'mean']
    print(dft[par].isnull().sum())
    return dft

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/ousama-88/prohack_dataset_fIZoqt7/master/train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/ousama-88/prohack_dataset_fIZoqt7/master/test.csv')

train = train[train['galaxy'] != 'NGC 5253']

test_years = test['galactic year'].value_counts()
test['nulls'] = test.isnull().sum(axis = 1)
gr_test = test.groupby('galactic year')['nulls'].mean()
test = test.drop(['nulls'], axis = 1)

train['nulls'] = train.isnull().sum(axis = 1)
gr_train = train.groupby('galactic year')['nulls'].mean()
train = train.drop(['nulls'], axis = 1)

gr2 = group_stat(train, ['galaxy'], 'y')

gr = group_stat(train, ['galactic year'], 'y')

df_galaxy = train['galaxy'].value_counts()
galaxys = list(df_galaxy.index)
cols = list(train.columns)
df_years = train['galactic year'].value_counts()
years = list(df_years.index)
years.sort()
    
test['y'] = 0
test = test.append(train, sort=False).reset_index(drop=True)

test.loc[test['y'] == 0, 'y'] = np.nan
gr = test.groupby('galaxy').mean()

test['y'] = np.log(test['y'])


test['Population using at least basic drinking-water services (%)'] = test['Population using at least basic drinking-water services (%)'] - 15
test.loc[(test['Population using at least basic drinking-water services (%)'] > 100), 'Population using at least basic drinking-water services (%)'] = 100


test.loc[test['Gross income per capita'] <= 0, 'Gross income per capita'] = np.nan
test['Gross income per capita'] = np.log(test['Gross income per capita'])

test['Population using at least basic sanitation services (%)'] = \
test['Population using at least basic sanitation services (%)'].max() - \
test['Population using at least basic sanitation services (%)'] 


list_log = ['Population using at least basic sanitation services (%)',
            'Mortality rate, under-five (per 1,000 live births)',
            'Mortality rate, infant (per 1,000 live births)',
            'Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64))',
            'Adolescent birth rate (births per 1,000 female creatures ages 15-19)',
            'Unemployment, total (% of labour force)',
            'Unemployment, youth (% ages 15–24)',
            'Mortality rate, female grown up (per 1,000 people)',
            'Mortality rate, male grown up (per 1,000 people)',
            'Infants lacking immunization, red hot disease (% of one-galactic year-olds)',
            'Infants lacking immunization, Combination Vaccine (% of one-galactic year-olds)',
            'Gross galactic product (GGP) per capita',
            'Natural resource depletion',
            'Maternal mortality ratio (deaths per 100,000 live births)',
            'Estimated gross galactic income per capita, male',
            'Estimated gross galactic income per capita, female',
            'Domestic credit provided by financial sector (% of GGP)',
            'Remittances, inflows (% of GGP)']

for par in list_log:
    test.loc[test[par] < 0, par] = np.nan
test[list_log] = np.log(test[list_log] + 0.1)


test.loc[(test['galactic year'] < 1000000), 'Intergalactic Development Index (IDI), Rank'] = \
test.loc[(test['galactic year'] < 1000000), 'Intergalactic Development Index (IDI), Rank'] + 15


##Reduction of all parameters to constants (average) by years
for col in cols[2:-1]:
    test = korrect_y(test, col)

test['Population, total calc'] = test['Population, ages 15–64 (millions)'] + test['Population, ages 65 and older (millions)'] + \
test['Population, ages 15–64 (millions)'] * test['Young age (0-14) dependency ratio (per 100 creatures ages 15-64)'] / 100

test['total dependency ratio calc'] = test['Population, ages 15–64 (millions)'] / \
(test['Population, total calc'] - test['Population, ages 15–64 (millions)'])

test['Gross galactic product (GGP) per capita calc'] = test['Gross galactic product (GGP), total'] / \
test['Population, total calc']

test['HDI'] = (test['existence expectancy index'] * test['Income Index'] * test['Education Index'])**(1/3)

test['Life expectancy index'] = (test['existence expectancy at birth'] - 20) / (85 - 20)
data = test[['Life expectancy index', 'existence expectancy index', 'existence expectancy at birth']]


##Gender Ratio Parameters
##Parameters of the ratio of indicators in half

test['Expected years of education ratio']  = test['Expected years of education, male (galactic years)'] /\
                                            test['Expected years of education, female (galactic years)']
test['Population with secondary education ratio']  = test['Population with at least some secondary education, male (% ages 25 and older)'] /\
                                                    test['Population with at least some secondary education, female (% ages 25 and older)']
test['Intergalactic Development Index ratio']  = test['Intergalactic Development Index (IDI), male'] /\
                                                test['Intergalactic Development Index (IDI), female']
test['Estimated gross galactic income per capita ratio']  = test['Estimated gross galactic income per capita, male'] /\
                                                           test['Estimated gross galactic income per capita, female']
test['Intergalactic Development Index ratio, Rank']  = test['Intergalactic Development Index (IDI), male, Rank'] /\
                                                      test['Intergalactic Development Index (IDI), female, Rank']
test['Labour force participation rate (% ages 15 and older) ratio']  = test['Labour force participation rate (% ages 15 and older), male'] /\
                                                                      test['Labour force participation rate (% ages 15 and older), female']
test['Life expectancy at birth ratio']  = test['Life expectancy at birth, male (galactic years)'] /\
                                         test['Life expectancy at birth, female (galactic years)']
test['Mortality rate ratio']  = test['Mortality rate, male grown up (per 1,000 people)'] /\
                               test['Mortality rate, female grown up (per 1,000 people)']
test['Expected to mean ratio']  = test['Expected years of education (galactic years)'] / test['Mean years of education (galactic years)']

test['Tourists per citizen'] = test['Intergalactic inbound tourists (thousands)'] / test['Population, total calc'] / 1000
test['Estimated gross galactic income per capita mean'] = (test['Estimated gross galactic income per capita, female'] + test['Estimated gross galactic income per capita, male']) / 2
test['Expected years of education (galactic years) mean'] = (test['Expected years of education, female (galactic years)'] + test['Expected years of education, male (galactic years)']) / 2
test['Intergalactic Development Index (IDI) mean'] = (test['Intergalactic Development Index (IDI), female'] + test['Intergalactic Development Index (IDI), male']) / 2
test['Intergalactic Development Index (IDI) mean, Rank'] = (test['Intergalactic Development Index (IDI), female, Rank'] + test['Intergalactic Development Index (IDI), male, Rank']) / 2
test['Labour force participation rate (% ages 15 and older) mean'] = (test['Labour force participation rate (% ages 15 and older), female'] + test['Labour force participation rate (% ages 15 and older), male']) / 2
test['Mean years of education (galactic years) mean'] = (test['Mean years of education, female (galactic years)'] + test['Mean years of education, male (galactic years)']) / 2
test['Mortality rate mean'] = (test['Mortality rate, male grown up (per 1,000 people)'] + test['Mortality rate, female grown up (per 1,000 people)']) / 2
test['Population with at least some secondary education mean'] = (test['Population with at least some secondary education, female (% ages 25 and older)'] + test['Population with at least some secondary education, male (% ages 25 and older)']) / 2
test['Private galaxy capital flows (% of GGP) per capita'] = test['Private galaxy capital flows (% of GGP)'] / test['Population, total calc']


cols_drop = ['Remittances, inflows (% of GGP)', 'Population, total (millions)', 'Population, total calc', 
             'Population, ages 65 and older (millions)', 'Intergalactic inbound tourists (thousands)',
             'Population, ages 15–64 (millions)', 'Population, under age 5 (millions)', 'existence expectancy at birth',
             'Estimated gross galactic income per capita, female', 'Estimated gross galactic income per capita, male',
             'Expected years of education, female (galactic years)', 'Expected years of education, male (galactic years)',
             'Intergalactic Development Index (IDI), female', 'Intergalactic Development Index (IDI), male',
             'Intergalactic Development Index (IDI), female, Rank', 'Intergalactic Development Index (IDI), male, Rank',
             'Mean years of education, female (galactic years)', 'Mean years of education, male (galactic years)',
             'Mortality rate, male grown up (per 1,000 people)', 'Mortality rate, female grown up (per 1,000 people)',
             'Population with at least some secondary education, female (% ages 25 and older)', 
             'Population with at least some secondary education, male (% ages 25 and older)',
             'Labour force participation rate (% ages 15 and older), female', 'Labour force participation rate (% ages 15 and older), male',
             'Outer Galaxies direct investment, net inflows (% of GGP)', 'Jungle area (% of total land area)',
             'Gross galactic product (GGP), total', 'Gross capital formation (% of GGP)', 'Total unemployment rate (female to male ratio)',
             'Gross fixed capital formation (% of GGP)','Gross enrolment ratio, primary (% of primary under-age population)',
             'Renewable energy consumption (% of total final energy consumption)', 'Private galaxy capital flows (% of GGP)']
test = test.drop(cols_drop, axis = 1)

test = movecol(test, cols_to_move=['y'], ref_col='Private galaxy capital flows (% of GGP) per capita', place='After')
cols = list(test.columns)
dft_galaxy = group_stat(test, ['galaxy'], 'y').sort_values(by = 'Par_mean')
galaxys = dft_galaxy['galaxy'].tolist()

scalerx = MinMaxScaler(feature_range = (1, 2))
scalery = MinMaxScaler(feature_range = (1, 2))

test['galactic year sc'] = scalerx.fit_transform(test['galactic year'].values.reshape(-1, 1))
test['y_start'] = 0
test['y_din'] = 0

for galaxy in galaxys:
    dfg = test.loc[test['galaxy'] == galaxy] 
    dfg = dfg.dropna(subset = ['y'])
    gy = dfg[dfg['galactic year'] < 1011030]['galactic year sc'].values
    y_din = dfg[dfg['galactic year'] < 1011030]['y'].values
    popt, pcov = curve_fit(func, gy, y_din, maxfev = 100000)    
    test.loc[test['galaxy'] == galaxy, 'y_din'] = popt[0]    
    test.loc[test['galaxy'] == galaxy, 'y_start'] = popt[1]
test = test.drop('galactic year sc', axis = 1)

i2 = 0
model = CatBoostRegressor(iterations = 10000,learning_rate = 0.05, depth = 4,
                                           l2_leaf_reg = 0.1, custom_metric = 'RMSE', eval_metric = 'RMSE', verbose = False)

from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(4)
test = test.sort_values(by = 'galactic year')
start = time.monotonic()
n = 4
for par in cols[2:-1]:
#    print(par, 'before', np.corrcoef(dft.loc[(~dft['y'].isnull()) & (~dft[par].isnull()), par], dft.loc[(~dft['y'].isnull()) & (~dft[par].isnull()), 'y'])[0, 1])
#    fig, ax = plt.subplots(figsize=(20, 20))
#    sns.distplot(dft.loc[~dft[par].isnull(), par], hist = True, fit=norm, kde = False, ax=ax)
    list_x = ['galactic year', 'Par1', 'y_start', 'y_din']
    test['Par1'] = np.random.randint(0, 10000, test.shape[0])
    data = test
    data = data.dropna(subset = [par])
    x = poly.fit_transform(test[list_x])
    y = test[par].copy()

    xa_sc = scalerx.fit_transform(x)
    x_sc = scalerx.transform(poly.transform(data[list_x]))
    y_sc = scalery.fit_transform(data[par].values.reshape(-1, 1))
    y_sc_r = y_sc.ravel()
    
    y_pred = np.zeros(xa_sc.shape[0])
    cv = KFold(n_splits=n, shuffle=True)
    splits = cv.split(x_sc)
    for train_fold_index, predict_fold_index in splits:
        eval_pool = Pool(x_sc[predict_fold_index], y_sc_r[predict_fold_index])
        model.fit(x_sc[train_fold_index], y_sc_r[train_fold_index], eval_set=eval_pool, early_stopping_rounds=50)
        y_pred += scalery.inverse_transform(model.predict(xa_sc).reshape(-1, 1)).ravel()
   
    
    y_pred = y_pred / n
    
    
    test[par] = y_pred
    test['par'] = y
    
    for galaxy in galaxys:
        spl = splrep(test[test['galaxy'] == galaxy]['galactic year'],test[test['galaxy'] == galaxy][par], k = 3, s = 150)
        y_pred = splev(test[test['galaxy'] == galaxy]['galactic year'], spl)
        test.loc[test['galaxy'] == galaxy, par] = y_pred 
end = time.monotonic()
test = test.sort_index() 

test = test.drop(['par', 'Par1', 'y_start', 'y_din'], axis = 1)


In [ ]:
test['break_year'] = test['galactic year'].apply(lambda x: 0 if x < 1011030 else 1)
test = movecol(test, cols_to_move=['y'], ref_col='break_year', place='After')
#cor_all = dft.corr()
#df = dft.loc[~dft['y'].isnull(), :]    

In [ ]:
test.to_csv('dft_medium.csv', index=False)



---



---

